In [ ]:
# Import necessary libraries
import sys
import os
import requests
import json
from pathlib import Path

# Add parent directory to path for imports
sys.path.append('..')

# Check if we're in the correct environment
print("Current working directory:", os.getcwd())
print("Python executable:", sys.executable)
print("Virtual environment active:", hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix))


In [ ]:
# Test importing our face analyzer
try:
    from app.utils.face_analyzer import analyze_face, MODEL_PATH
    from app.models.profile import Profile
    print("✅ Successfully imported face analyzer components!")
    print(f"Model path expected at: {MODEL_PATH}")
    
    # Check if model file exists
    if os.path.exists(MODEL_PATH):
        print("✅ dlib model file found!")
    else:
        print("❌ dlib model file missing!")
        print("Download it with:")
        print("mkdir -p models")
        print("wget https://github.com/davisking/dlib-models/raw/master/shape_predictor_68_face_landmarks.dat -P models/")
        
except ImportError as e:
    print(f"❌ Import error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")


In [ ]:
# Test face analyzer with invalid input (should raise ValueError)
try:
    result = analyze_face(b"not an image")
    print("❌ Should have raised ValueError")
except ValueError as e:
    print(f"✅ Correctly handled invalid input: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

# Test with minimal valid image (1x1 black pixel PNG)
import base64
minimal_png = base64.b64decode(
    b"iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR4nGNgYAAAAAMAASsJTYQAAAAASUVORK5CYII="
)

try:
    result = analyze_face(minimal_png)
    print("❌ Should have raised ValueError (no face in 1x1 image)")
except ValueError as e:
    print(f"✅ Correctly detected no face in minimal image: {e}")
except RuntimeError as e:
    print(f"⚠️  Model file issue (expected if not downloaded): {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")


In [ ]:
# Test server connectivity
API_BASE = "http://127.0.0.1:8000"

try:
    # Test basic connectivity
    response = requests.get(f"{API_BASE}/")
    if response.status_code == 200:
        print("✅ Server is running!")
        print(f"Response: {response.json()}")
    else:
        print(f"❌ Server responded with status {response.status_code}")
        
    # Test ping endpoint
    response = requests.get(f"{API_BASE}/v1/ping")
    if response.status_code == 200:
        print("✅ V1 API is accessible!")
        print(f"Ping response: {response.json()}")
    else:
        print(f"❌ V1 API responded with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to server. Make sure to run:")
    print("uvicorn app.main:app --reload")
except Exception as e:
    print(f"❌ Unexpected error: {e}")


In [ ]:
# Test create-profile endpoint with invalid data
try:
    # Test with invalid file data
    files = {'file': ('test.txt', b'not an image', 'text/plain')}
    response = requests.post(f"{API_BASE}/v1/create-profile", files=files)
    
    print(f"Status code: {response.status_code}")
    print(f"Response: {response.json()}")
    
    if response.status_code == 400:
        print("✅ Correctly rejected invalid image data")
    elif response.status_code == 500:
        print("⚠️  Server error (likely missing dlib model file)")
    else:
        print(f"❌ Unexpected status code: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ Server not running")
except Exception as e:
    print(f"❌ Error testing endpoint: {e}")


In [ ]:
# Test with a real face image (if available)
def test_with_face_image(image_path):
    """Test the create-profile endpoint with a real face image."""
    if not os.path.exists(image_path):
        print(f"❌ Image file not found: {image_path}")
        return
    
    try:
        with open(image_path, 'rb') as f:
            files = {'file': (os.path.basename(image_path), f, 'image/jpeg')}
            response = requests.post(f"{API_BASE}/v1/create-profile", files=files)
        
        print(f"Status code: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Successfully created facial profile!")
            print(f"Description: {result.get('description', 'N/A')}")
            print(f"Eye distance: {result.get('eye_distance', 'N/A'):.2f}px")
            print(f"Yaw: {result.get('yaw', 'N/A'):.2f}")
            print(f"Number of landmarks: {len(result.get('landmarks', []))}")
            
            # Show first few landmarks
            landmarks = result.get('landmarks', [])
            if landmarks:
                print(f"First 5 landmarks: {landmarks[:5]}")
                
        elif response.status_code == 400:
            print(f"❌ Bad request: {response.json()}")
        elif response.status_code == 500:
            print(f"⚠️  Server error: {response.json()}")
        else:
            print(f"❌ Unexpected response: {response.json()}")
            
    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage (uncomment and provide a real image path)
# test_with_face_image("path/to/your/face_image.jpg")

print("To test with a real image:")
print("1. Download the dlib model file")
print("2. Place a face image in the project directory")
print("3. Uncomment the test_with_face_image call above")


In [ ]:
# Run tests programmatically
import subprocess

try:
    # Run pytest with verbose output
    result = subprocess.run(
        ["python", "-m", "pytest", "../app/tests/", "-v"], 
        capture_output=True, 
        text=True,
        cwd=".."
    )
    
    print("STDOUT:")
    print(result.stdout)
    
    if result.stderr:
        print("STDERR:")
        print(result.stderr)
    
    print(f"Return code: {result.returncode}")
    
    if result.returncode == 0:
        print("✅ All tests passed!")
    else:
        print("❌ Some tests failed")
        
except Exception as e:
    print(f"❌ Error running tests: {e}")
    print("Try running manually: pytest -v")
